In [1]:
!! pip install python-geohash
!! pip install geohash

['Requirement already satisfied: geohash in /usr/local/lib/python3.6/dist-packages (1.0)',
 'Requirement already satisfied: docutils>=0.3 in /usr/local/lib/python3.6/dist-packages (from geohash) (0.14)']

In [0]:
# from tensorflow.python.client import device_lib
# print(str(device_lib.list_local_devices()))

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pickle import Pickler, Unpickler
import Geohash
from sklearn.metrics import accuracy_score

In [4]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [0]:
root_path = "gdrive/My Drive/Colab Notebooks/"

In [0]:
#Importando Dataset
# df = Unpickler(open('./df_rdo_categ.sav', 'rb')).load()
df1 = Unpickler(open(root_path + "dataset/rdo1.sav", 'rb')).load()
df2 = Unpickler(open(root_path + "/dataset/rdo2.sav", 'rb')).load()
df3 = Unpickler(open(root_path + "/dataset/rdo3.sav", 'rb')).load()
df = pd.concat([df1, df2, df3])

In [0]:
# df1 = df.iloc[:586000, :]
# df2 = df.iloc[586000:1172000, :]
# df3 = df.iloc[1172000:, :]

# f = open('./rdo1.sav', 'wb')
# Pickler(f).dump(df1)
# f.close()

# f = open('./rdo2.sav', 'wb')
# Pickler(f).dump(df2)
# f.close()

# f = open('./rdo3.sav', 'wb')
# Pickler(f).dump(df3)
# f.close()

In [0]:
# df = df.drop(columns=['LATITUDE', 'LONGITUDE'])
# df.head()

In [0]:
X = df.drop(columns=['GEOHASH']).values
y = df[['GEOHASH']]
y = pd.get_dummies(y, columns = ['GEOHASH'])
y = y.values

In [0]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [0]:
## Separando as coordenadas para plotar num mapa depois e comparar com as geohashes

X_teste_coordinates = X_test[:, :2]
X_train_coordinates = X_train[:, :2]

In [0]:
# Removendo colunas das coordenadas, pois elas não farão parte do treino ou teste. No lugar, teremos as geohashes
X_test = X_test[:, 2:]
X_train = X_train[:, 2:]

In [0]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [0]:
# Necessário para pegar a categoriar, já que ainda está no formato de dummy

y_test = np.argmax(y_test, axis=1)

## Construindo classificador

In [14]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

Using TensorFlow backend.


In [15]:
len(df['GEOHASH'].unique())

282

In [0]:
# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 21))
classifier.add(Dropout(rate = 0.1))

# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'sigmoid'))
classifier.add(Dropout(rate = 0.1))

# Adding the third hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'sigmoid'))
classifier.add(Dropout(rate = 0.1))

# Adding the output layer
classifier.add(Dense(units = 282, kernel_initializer = 'uniform', activation = 'relu'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [108]:
# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size = 100, epochs = 20)

Epoch 1/20
1236231/1236231 [==============================] - 51s 41us/step - loss: 0.0353 - acc: 0.9965
Epoch 2/20
1236231/1236231 [==============================] - 49s 40us/step - loss: 0.0351 - acc: 0.9965
Epoch 3/20
1236231/1236231 [==============================] - 49s 40us/step - loss: 0.0351 - acc: 0.9965
Epoch 4/20
1236231/1236231 [==============================] - 50s 40us/step - loss: 0.0351 - acc: 0.9965
Epoch 5/20
1236231/1236231 [==============================] - 50s 40us/step - loss: 0.0351 - acc: 0.9965
Epoch 6/20
1236231/1236231 [==============================] - 49s 40us/step - loss: 0.0352 - acc: 0.9965
Epoch 7/20
1236231/1236231 [==============================] - 49s 40us/step - loss: 0.0355 - acc: 0.9965
Epoch 8/20
1236231/1236231 [==============================] - 49s 40us/step - loss: 0.0358 - acc: 0.9965
Epoch 9/20
1236231/1236231 [==============================] - 49s 40us/step - loss: 0.0358 - acc: 0.9965
Epoch 10/20
1236231/1236231 [==========================

In [0]:
y_pred = classifier.predict(X_test)

In [0]:
y_pred = np.argmax(y_pred, axis=1)

In [111]:
y_pred

array([142, 142, 142, ..., 142, 142, 142])

In [112]:
accuracy_score(y_test, y_pred)

0.09836848403401949